In [13]:
import nltk
from __future__ import print_function
from nltk.classify import apply_features

# A sense of machine learning

We will try a simple example of text classification. The examples below come from the [excellent NLTK book](http://www.nltk.org/book/ch06.html)

Detecting patterns is a central part of Natural Language Processing. Words ending in "-ed" tend to be past tense verbs. Frequent use of "will" is indicative of news text. These observable patterns — word structure and word frequency — happen to correlate with particular aspects of meaning, such as tense and topic. But how did we know where to start looking, which aspects of form to associate with which aspects of meaning?

In this notebook, we tackle the following questions:

* How can we identify particular features of language data that are salient for classifying it?
* How can we construct models of language that can be used to perform language processing tasks automatically?
* What can we learn about language from these models?


## Supervised Classification

Classification is the task of choosing the correct class label for a given input. In basic classification tasks, each input is considered in isolation from all other inputs, and the set of labels is defined in advance. Some examples of classification tasks are:

* Deciding whether an email is spam or not
* Deciding what the topic of a news article is, from a fixed list of topic areas such as "sports," "technology," and "politics"
* Deciding whether a given occurrence of the word *bank* is used to refer to a river bank, a financial institution, the act of tilting to the side, or the act of depositing something in a financial institution

A classifier is called *supervised* if it is built based on training corpora containing the correct label for each input. The framework used by supervised classification is shown below:

<img src="supervised-classification.png" alt="" style="width: 400px;"/>

* (a) During training, a feature extractor is used to convert each input value to a feature set. These feature sets, which capture the basic information about each input that should be used to classify it, are discussed in the next section. Pairs of feature sets and labels are fed into the machine learning algorithm to generate a model. 
* (b) During prediction, the same feature extractor is used to convert unseen inputs to feature sets. These feature sets are then fed into the model, which generates predicted labels.


## Example: Gender Identification

In the rest of this notebook, we will look at how  a supervised classifier can be employed to identify the gender of a proper name.

Male and female names have some distinctive characteristics. Names ending in a, e and i are likely to be female, while names ending in k, o, r, s and t are likely to be male. Let's build a classifier to model these differences more precisely.

The first step in creating a classifier is deciding what features of the input are relevant, and how to encode those features. For this example, we'll start by just looking at the final letter of a given name. The following `gender_features()` feature extractor function builds a dictionary containing relevant information about a given name:    

In [14]:
def gender_features(word):
    return {'last_letter': word[-1]}
gender_features('Shrek')

{'last_letter': 'k'}

The returned dictionary, known as a feature set, maps from feature names to their values. Feature names are case-sensitive strings that typically provide a short human-readable description of the feature, as in the example 'last_letter'. Feature values are values with simple types, such as booleans, numbers, and strings.

Now that we've defined a feature extractor, we need to prepare a list of examples and corresponding class labels. 

In [15]:
import nltk
from nltk.corpus import names
print('male names:', names.words('male.txt')[:7])

labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
    [(name, 'female') for name in names.words('female.txt')])

import random
random.shuffle(labeled_names)

male names: [u'Aamir', u'Aaron', u'Abbey', u'Abbie', u'Abbot', u'Abbott', u'Abby']


Next, we use the `gender_features()` feature extractor to process the names data, and divide the resulting list of feature sets into a training set and a test set. The training set is used to train a new "naive Bayes" classifier.  

In [16]:
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

classifier = nltk.NaiveBayesClassifier.train(train_set)
print('accuracy: ', nltk.classify.accuracy(classifier, test_set))

# test it out on some names that did not appear in its training data:   
print('Neo:      ', classifier.classify(gender_features('Neo')))
print('Trinity:  ', classifier.classify(gender_features('Trinity')))

accuracy:  0.768
Neo:       male
Trinity:   female


Observe that these character names from The Matrix are correctly classified. Although this science fiction movie is set in 2199, it still conforms with our expectations about names and genders. We can systematically evaluate the classifier on a much larger quantity of unseen data:    

Finally, we can examine the classifier to determine which features it found most effective for distinguishing the names' genders:

In [17]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = u'a'           female : male   =     34.3 : 1.0
             last_letter = u'k'             male : female =     32.0 : 1.0
             last_letter = u'f'             male : female =     16.8 : 1.0
             last_letter = u'p'             male : female =     12.7 : 1.0
             last_letter = u'v'             male : female =     10.6 : 1.0


This listing shows that the names in the training set that end in `a` are female 33 times more often than they are male, but names that end in `k` are male 32 times more often than they are female. These ratios are known as likelihood ratios, and can be useful for comparing different feature-outcome relationships.

## Your Turn

Modify the `gender_features()` function to provide the classifier with features encoding the length of the name, its first letter, and any other features that seem like they might be informative. Retrain the classifier with these new features, and test its accuracy.

---

##  Choosing The Right Features

Selecting relevant features and deciding how to encode them for a learning method can have an enormous impact on the learning method's ability to extract a good model. Much of the interesting work in building a classifier is deciding what features might be relevant, and how we can represent them. Although it's often possible to get decent performance by using a fairly simple and obvious set of features, there are usually significant gains to be had by using carefully constructed features based on a thorough understanding of the task at hand.

Typically, feature extractors are built through a process of trial-and-error, guided by intuitions about what information is relevant to the problem. It's common to start with a "kitchen sink" approach, including all the features that you can think of, and then checking to see which features actually are helpful. We take this approach for `gender_features2()` below.

However, there are usually limits to the number of features that you should use with a given learning algorithm — if you provide too many features, then the algorithm will have a higher chance of relying on idiosyncrasies of your training data that don't generalize well to new examples. This problem is known as **overfitting**, and can be especially problematic when working with small training sets. For example, if we train a naive Bayes classifier using the feature extractor below, it has too many features and will overfit the relatively small training set, resulting in a system whose accuracy is about 1% lower than the accuracy of a classifier that only pays attention to the final letter of each name:

In [18]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features
    
print('features2 for John', gender_features2('John'))

train_set2 = apply_features(gender_features2, labeled_names[500:])
test_set2 = apply_features(gender_features2, labeled_names[:500])
classifier2 = nltk.NaiveBayesClassifier.train(train_set2)
print('\nfeatures2 accuracy', nltk.classify.accuracy(classifier2, test_set2))

features2 for John {'count(u)': 0, 'has(d)': False, 'count(b)': 0, 'count(w)': 0, 'has(b)': False, 'count(l)': 0, 'count(q)': 0, 'count(n)': 1, 'has(j)': True, 'count(s)': 0, 'count(h)': 1, 'has(h)': True, 'has(y)': False, 'count(j)': 1, 'has(f)': False, 'has(o)': True, 'count(x)': 0, 'has(m)': False, 'count(z)': 0, 'has(k)': False, 'has(u)': False, 'count(d)': 0, 'has(s)': False, 'count(m)': 0, 'count(f)': 0, 'has(q)': False, 'has(w)': False, 'has(e)': False, 'has(z)': False, 'count(t)': 0, 'count(c)': 0, 'has(c)': False, 'has(x)': False, 'count(v)': 0, 'has(a)': False, 'last_letter': 'n', 'has(v)': False, 'count(p)': 0, 'count(o)': 1, 'first_letter': 'j', 'has(i)': False, 'count(i)': 0, 'has(r)': False, 'has(g)': False, 'count(k)': 0, 'count(y)': 0, 'has(n)': True, 'has(l)': False, 'count(e)': 0, 'has(t)': False, 'count(g)': 0, 'count(r)': 0, 'count(a)': 0, 'has(p)': False}

features2 accuracy 0.744


## Error analysis

Once an initial set of features has been chosen, a very productive method for refining the feature set is error analysis. First, we select a development set, containing the corpus data for creating the model. This development set is then subdivided into the training set and the dev-test set.   

In [19]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

The training set is used to train the model, and the dev-test set is used to perform error analysis. The test set serves in our final evaluation of the system. For reasons discussed below, it is important that we employ a separate dev-test set for error analysis, rather than just using the test set. The division of the corpus data into different subsets is shown below:

<img src="corpus-org.png" alt="" style="width: 400px;"/>

Organization of corpus data for training supervised classifiers. The corpus data is divided into two sets: the development set, and the test set. The development set is often further subdivided into a training set and a dev-test set.
Having divided the corpus into appropriate datasets, we train a model using the training set [1], and then run it on the dev-test set [2].    

In [20]:
train_set3 = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set3 = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set3 = [(gender_features(n), gender) for (n, gender) in test_names]
classifier3 = nltk.NaiveBayesClassifier.train(train_set3) #[1]
print(nltk.classify.accuracy(classifier3, devtest_set3)) #[2]

0.755


Using the dev-test set, we can generate a list of the errors that the classifier makes when predicting name genders.

We can then examine individual error cases where the model predicted the wrong label, and try to determine what additional pieces of information would allow it to make the right decision (or which existing pieces of information are tricking it into making the wrong decision). The feature set can then be adjusted accordingly. The names classifier that we have built generates about 100 errors on the dev-test corpus:

In [21]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )
        
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Aeriel                        
correct=female   guess=male     name=Aeriell                       
correct=female   guess=male     name=Ag                            
correct=female   guess=male     name=Aigneis                       
correct=female   guess=male     name=Aimil                         
correct=female   guess=male     name=Alisun                        
correct=female   guess=male     name=Alleen                        
correct=female   guess=male     name=Anabel                        
correct=female   guess=male     name=Anet                          
correct=female   guess=male     name=Angel                         
correct=female   guess=male     name=Ann                           
correct=female   guess=male     name=Annabell                      
correct=female   guess=male     name=Ardis                         
correct=female   guess=male     name=Ariel                         
correct=female   guess=male     name=Bab        

Looking through this list of errors makes it clear that *some suffixes that are more than one letter can be indicative of name genders*. For example, names ending in yn appear to be predominantly female, despite the fact that names ending in n tend to be male; and names ending in ch are usually male, even though names that end in h tend to be female. We therefore adjust our feature extractor to include features for two-letter suffixes:    

In [22]:
def gender_features4(word):
    return {'suffix1': word[-1:], 'suffix2': word[-2:]}

Rebuilding the classifier with the new feature extractor, we see that the performance on the dev-test dataset improves by almost 2 percentage points:

In [23]:
train_set4 = [(gender_features4(n), gender) for (n, gender) in train_names]
devtest_set4 = [(gender_features4(n), gender) for (n, gender) in devtest_names]
classifier4 = nltk.NaiveBayesClassifier.train(train_set4)
print(nltk.classify.accuracy(classifier4, devtest_set4))

0.783


This error analysis procedure can then be repeated, checking for patterns in the errors that are made by the newly improved classifier. Each time the error analysis procedure is repeated, we should select a different dev-test/training split, to ensure that the classifier does not start to reflect idiosyncrasies in the dev-test set.

But once we've used the dev-test set to help us develop the model, we can no longer trust that it will give us an accurate idea of how well the model would perform on new data. It is therefore important to keep the test set separate, and unused, until our model development is complete. At that point, we can use the test set to evaluate how well our model will perform on new input values.